In [ ]:
import urllib.request
import pandas as pd
import json
from tinydb import TinyDB, Query
import pprint as pp

download_urls = [
'https://api.covidtracking.com/v2/us/daily.json'
]

for i in download_urls:
    webURL = urllib.request.urlopen(i)
    data = webURL.read()
    encoding = webURL.info().get_content_charset('utf-8')
    json_o=json.loads(data.decode(encoding))
#json_o is a python dictionary
#we need to flatten the data if we want to manipulate it using pandas

meta_d=json_o['meta'] 

#meta_d is seperated on the meta key, since this is an info table there is little reason to manipulate it,
#but we can flatten it if we want to and then transpose it but I find it easier to just print the values

#print(meta_d)

#We have created a local NoSQL DB in the create_tinydb section
#normalise data based on feature key

df_data=pd.json_normalize(json_o,'data')
cols=list(df_data)

#convert to dataframe

df_data2= pd.DataFrame(df_data, columns=cols)

#Do something with data using dataframe
#fix nulls

df_data2.fillna(0, inplace=True)
               
#quick check
#df_data2.head(10)
                
Dict_data=df_data2.to_dict(orient='dict')

#convert back to dict and check for nulls
#pp.pprint(Dict_data)

#insert in to nosqldb

db_meta.insert(meta_d)
db_data1.insert(Dict_data)


